Reset stored data

In [6]:
%reset -f

Imports

In [7]:
import plotly.graph_objects as go
import numpy as np
import os
import plotly.io as pio
import sympy as sp

from sympy import symbols, Matrix, eye, det, Eq, solve, latex
from IPython.display import display, Markdown
from scipy.optimize import minimize
from lmfit import Model

# Data

In [104]:
# Data for uniaxial test
uniaxial_test_data = [
    ["Eng. Strain [%]", "Eng. Stress [MPa]"],
    [0, 0],
    [30, 8],
    [60, 15],
    [90, 20],
    [120, 28],
    [150, 43],
    [180, 65],
    [210, 92],
    [240, 116],
    [270, 171],
    [300, 220]
]

# Data for equibiaxial test
equibiaxial_test_data = [
    ["Eng. Strain [%]", "Eng. Stress [MPa]"],
    [0, 0],
    [30, 18],
    [60, 30],
    [90, 47],
    [120, 59],
    [150, 85],
    [180, 105],
    [210, 140],
    [240, 176],
    [270, 231],
    [300, 307]
]

# Data for planar test
planar_test_data = [
    ["Eng. Strain [%]", "Eng. Stress [MPa]"],
    [0, 0],
    [30, 12],
    [60, 18],
    [90, 26],
    [120, 35],
    [150, 50],
    [180, 69],
    [210, 95],
    [240, 132],
    [270, 169],
    [300, 227]
]

# Tasks

## Task 1

<b>Plot the experimental data points </b> for each test in the same coordinate system. Use stretch for the horizontal 
axis and the nominal stress for the vertical axis. The plot range for stretch must be $\lambda =$ 1 … 4. The plot range for the 
nominal stress must be $𝑃 = 0$ … $𝑃_{max}$, where $𝑃_{max}$ is the maximum measured nominal stress in the equibiaxial test. You 
can connect the data point with lines, but use dots to indicate the particular data points.

Plot for epsilon

In [105]:
# Extract data for each test
uniaxial_x = [row[0] for row in uniaxial_test_data[1:]]
uniaxial_y = [row[1] for row in uniaxial_test_data[1:]]

equibiaxial_x = [row[0] for row in equibiaxial_test_data[1:]]
equibiaxial_y = [row[1] for row in equibiaxial_test_data[1:]]

planar_x = [row[0] for row in planar_test_data[1:]]
planar_y = [row[1] for row in planar_test_data[1:]]

# Create traces for each test with lines and markers
trace_uniaxial = go.Scatter(x=uniaxial_x, y=uniaxial_y, mode='lines+markers', name='Uniaxial Test')
trace_equibiaxial = go.Scatter(x=equibiaxial_x, y=equibiaxial_y, mode='lines+markers', name='Equibiaxial Test')
trace_planar = go.Scatter(x=planar_x, y=planar_y, mode='lines+markers', name='Planar Test')

# Create layout
layout = go.Layout(
    title='Stress-Strain Curves for Different Tests',
    xaxis=dict(title='Eng. Strain [%]'),
    yaxis=dict(title='Eng. Stress [MPa]')
)

# Create figure
fig = go.Figure(data=[trace_uniaxial, trace_equibiaxial, trace_planar], layout=layout)

# Show the plot
fig.show()

Plot for Lambda

In [106]:
# Calculate lambda values
calculate_lambda = lambda strain: strain / 100 + 1

# Calculate lambda values for each test
lambda_uniaxial = [calculate_lambda(row[0]) for row in uniaxial_test_data[1:]]
lambda_equibiaxial = [calculate_lambda(row[0]) for row in equibiaxial_test_data[1:]]
lambda_planar = [calculate_lambda(row[0]) for row in planar_test_data[1:]]

# Extract y values for each test
uniaxial_y = [row[1] for row in uniaxial_test_data[1:]]
equibiaxial_y = [row[1] for row in equibiaxial_test_data[1:]]
planar_y = [row[1] for row in planar_test_data[1:]]

# Create traces for each test with lines and markers
trace_uniaxial = go.Scatter(x=lambda_uniaxial, y=uniaxial_y, mode='lines+markers', name='Uniaxial Test')
trace_equibiaxial = go.Scatter(x=lambda_equibiaxial, y=equibiaxial_y, mode='lines+markers', name='Equibiaxial Test')
trace_planar = go.Scatter(x=lambda_planar, y=planar_y, mode='lines+markers', name='Planar Test')

# Create layout
layout = go.Layout(
    xaxis=dict(title='Stretch [1]', tickmode='array', tickvals=[1, 1.5, 2, 2.5, 3, 3.5, 4]),
    yaxis=dict(title='Eng. Stress [MPa]')
)

# Create figure
fig = go.Figure(data=[trace_uniaxial, trace_equibiaxial, trace_planar], layout=layout)

# Show the plot
fig.show()

fig.write_html("plot_task1.html")
pio.write_image(fig, "plot_task1.pdf", width=1400, height=600)
fig.write_image("plot_task1.jpg", width=1400, height=600)

## Task 2

Obtain the material parameters by <b>fitting the model only to the uniaxial data </b>. Consequently, in the calculation 
of the quality function use only the uniaxial data. The parameter-fitting is acceptable if $𝑄_𝑈$ < 5 %. Report the values of 
the fitted material parameters and $𝑄_𝑈$.

In [107]:
# Experimental data
lambda_exp_uniaxial = np.array(lambda_uniaxial)
lambda_exp_equibiaxial = np.array(lambda_equibiaxial)
lambda_exp_planar = np.array(lambda_planar)

P_exp_uniaxial = np.array(uniaxial_y)
P_exp_equibiaxial = np.array(equibiaxial_y)
P_exp_planar = np.array(planar_y)

# Define the W function
def W(lambdas, mu1, mu2, alpha1, alpha2):
    term1 = 2 * mu1 / alpha1**2 * (lambdas[0]**alpha1 + lambdas[1]**alpha1 + lambdas[2]**alpha1 - 3)
    term2 = 2 * mu2 / alpha2**2 * (lambdas[0]**alpha2 + lambdas[1]**alpha2 + lambdas[2]**alpha2 - 3)
    return term1 + term2

# Define the partial derivative functions
def P_U(lambdas, mu1, mu2, alpha1, alpha2):
    term1 = 2 * mu1 / alpha1 * (lambdas**(alpha1-1) - lambdas**((-alpha1/2) - 1))
    term2 = 2 * mu2 / alpha2 * (lambdas**(alpha2-1) - lambdas**((-alpha2/2) - 1))
    return term1 + term2

def P_B(lambdas, mu1, mu2, alpha1, alpha2):
    term1 = 2 * mu1 / alpha1 * (lambdas**(alpha1-1) - lambdas**((-2*alpha1) - 1))
    term2 = 2 * mu2 / alpha2 * (lambdas**(alpha2-1) - lambdas**((-2*alpha2) - 1))
    return term1 + term2

def P_P(lambdas, mu1, mu2, alpha1, alpha2):
    term1 = 2 * mu1 / alpha1 * (lambdas**(alpha1-1) - lambdas**((-alpha1) - 1))
    term2 = 2 * mu2 / alpha2 * (lambdas**(alpha2-1) - lambdas**((-alpha2) - 1))
    return term1 + term2

def quality_function(params, lambdas, P_exp, P_function):
    mu1, mu2, alpha1, alpha2 = params
    P_sim = P_function(lambdas, mu1, mu2, alpha1, alpha2)
    error = (P_exp - P_sim) / (P_exp + 1e-10)  
    return np.sqrt(np.mean(error**2))

# Function to calculate stress using optimal parameters
def calculate_stress_uniaxial(lambdas, mu1, mu2, alpha1, alpha2):
    P_sim = P_U(lambdas, mu1, mu2, alpha1, alpha2)
    return P_sim

def calculate_stress_equibiaxial(lambdas, mu1, mu2, alpha1, alpha2):
    P_sim = P_B(lambdas, mu1, mu2, alpha1, alpha2)
    return P_sim

def calculate_stress_planar(lambdas, mu1, mu2, alpha1, alpha2):
    P_sim = P_P(lambdas, mu1, mu2, alpha1, alpha2)
    return P_sim

# Fitting function
def fit_function(lambdas, mu1, mu2, alpha1, alpha2):
    return calculate_stress_uniaxial(lambdas, mu1, mu2, alpha1, alpha2)

In [109]:
min_error_value = float('inf')
best_method = None
best_params = None

# Define the optimization methods
methods = ['Nelder-Mead', 'BFGS', 'L-BFGS-B', 'Powell', 'COBYLA']

# Initial guess
mu1 = 0.1
mu2 = 0.1
alpha1 = -2
alpha2 = 1

# Initial parameters for optimization
initial_params = [mu1, mu2, alpha1, alpha2]

model = Model(fit_function)
model.set_param_hint('mu1', value=mu1)
model.set_param_hint('mu2', value=mu2)
model.set_param_hint('alpha1', value=alpha1)
model.set_param_hint('alpha2', value=alpha2)

# Loop through each optimization method
for method in methods:
    max_iter = 15000
    
    # Perform optimization with increased iterations using P_U
    result_optimize = minimize(quality_function, initial_params, args=(lambda_exp_uniaxial, P_exp_uniaxial, P_U), method=method, options={'maxiter': max_iter})
    optimal_params_P_U = result_optimize.x
    
    # Calculate error value for P_U using quality_function
    error_value_P_U = quality_function(optimal_params_P_U, lambda_exp_uniaxial, P_exp_uniaxial, P_U) * 100
    
    # Update minimum error value and best method for P_U
    if error_value_P_U < min_error_value:
        min_error_value = error_value_P_U
        best_method = method
        best_params = optimal_params_P_U

    # Print error value for P_U
    print(f"\nOptimal Parameters ({method}, P_U): {optimal_params_P_U}")
    print(f"Error Value ({method}, P_U): {error_value_P_U:.4f} % with {max_iter} iterations")

# Set the maximum number of iterations for lmfit
max_iter_lmfit = 15000

# Fit the model to uniaxial data using lmfit with increased iterations
result_lmfit = model.fit(P_exp_uniaxial, lambdas=lambda_exp_uniaxial, method='leastsq', iter_cb=None, max_nfev=max_iter_lmfit)
optimal_params_fit_lmfit = [result_lmfit.best_values[f] for f in ['mu1', 'mu2', 'alpha1', 'alpha2']]

# Calculate error value for lmfit method using quality_function
error_value_lmfit = quality_function(optimal_params_fit_lmfit, lambda_exp_uniaxial, P_exp_uniaxial, P_U) * 100
if error_value_lmfit < min_error_value:
    min_error_value = error_value_lmfit
    best_method = 'lmfit'
    best_params = optimal_params_fit_lmfit

print(f"\nOptimal Parameters (lmfit, P_U): {optimal_params_fit_lmfit}")
print(f"Error Value (lmfit, P_U): {error_value_lmfit:.4f} % with {max_iter_lmfit} iterations")
    
# Print the best method
print(f"\nBest Method: {best_method} (Error Value: {min_error_value:.4f} %)")
print(f"Optimal Parameters for the Best Method (P_U): {best_params}")


Optimal Parameters (Nelder-Mead, P_U): [10.55439559  2.1012538  -0.77321806  4.99867097]
Error Value (Nelder-Mead, P_U): 4.0832 % with 15000 iterations

Optimal Parameters (BFGS, P_U): [10.55441842  2.10122789 -0.77316348  4.99868234]
Error Value (BFGS, P_U): 4.0832 % with 15000 iterations

Optimal Parameters (L-BFGS-B, P_U): [10.55445786  2.10124775 -0.77327501  4.99867407]
Error Value (L-BFGS-B, P_U): 4.0832 % with 15000 iterations

Optimal Parameters (Powell, P_U): [15.66362192 -2.79230196 -7.8670914  -7.86616575]
Error Value (Powell, P_U): 10.7635 % with 15000 iterations

Optimal Parameters (COBYLA, P_U): [10.17868332  2.08433767 -0.35084183  5.00342576]
Error Value (COBYLA, P_U): 4.1048 % with 15000 iterations

Optimal Parameters (lmfit, P_U): [5.057573082318104, 9.432191162038052, -9.66295228421433, -1.8570238702943205]
Error Value (lmfit, P_U): 4.4795 % with 15000 iterations

Best Method: BFGS (Error Value: 4.0832 %)
Optimal Parameters for the Best Method (P_U): [10.55441842  2

In [110]:
# Plot the experimental and simulated uniaxial stress
fig = go.Figure()

# Experimental data
fig.add_trace(go.Scatter(x=lambda_exp_uniaxial, y=P_exp_uniaxial, mode='lines+markers', name='Experimental (Uniaxial)', line=dict(color='red', dash='dash')))

# Simulated data using the best method
simulated_uniaxial_stress = calculate_stress_uniaxial(lambda_exp_uniaxial, *best_params)
fig.add_trace(go.Scatter(x=lambda_exp_uniaxial, y=simulated_uniaxial_stress, mode='lines+markers', name=f'Simulated Uniaxial', line=dict(color='blue')))

# Latex annotations for parameters and quality function
latex_annotations_mu1 = f"$\\mu_1: {best_params[0]:.4f}$"
latex_annotations_mu2 = f"$\\mu_2: {best_params[1]:.4f}$"
latex_annotations_alpha1 = f"$\\alpha_1: {best_params[2]:.4f}$"
latex_annotations_alpha2 = f"$\\alpha_2: {best_params[3]:.4f}$"

fig.update_layout(
    showlegend=True,
    plot_bgcolor='white',  # Set background color to white
    xaxis=dict(title='Stretch [1]', gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='black'),  # Set grid color to lightgrey, x-axis line to black
    yaxis=dict(title='Engineering stress [MPa]', gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='black'),  # Set grid color to lightgrey, y-axis line to black
    annotations=[
        dict(text=latex_annotations_mu1, x=0.07, y=0.97, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
        dict(text=latex_annotations_mu2, x=0.20, y=0.97, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
        dict(text=latex_annotations_alpha1, x=0.07, y=0.89, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
        dict(text=latex_annotations_alpha2, x=0.20, y=0.89, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
    ],
    shapes=[
        dict(
            type='rect',
            xref='paper',
            yref='paper',
            x0=0.055,
            y0=0.82,
            x1=0.315,
            y1=1,
            fillcolor='white',
            opacity=1,
            layer='below',
            line=dict(color='black', width=2)
        ),
    dict(
            type='rect',
            xref='paper',
            yref='paper',
            x0=0.055,
            y0=0.75,
            x1=0.315,
            y1=0.82,
            fillcolor='white',
            opacity=1,
            layer='below',
            line=dict(color='black', width=2)
        ),
    ]
)

fig.add_annotation(
    text=f"Quality: {error_value_P_U:.4f} %",
    x=0.11, y=0.81,
    xref='paper', yref='paper',
    showarrow=False,
    align='left',
    font=dict(family='Arial, sans-serif')
)

# Show the plot
fig.show()

In [111]:
output_folder = "Cont_mecha_2HW"

# Save the equibiaxial plot as HTML, PDF, and JPG
uniaxial_html_path = os.path.join(output_folder, "uniaxial_plot.html")
uniaxial_pdf_path = os.path.join(output_folder, "uniaxial_plot.pdf")
uniaxial_jpg_path = os.path.join(output_folder, "uniaxial_plot.jpg")

fig.write_html(uniaxial_html_path)
pio.write_image(fig, uniaxial_pdf_path, width=1000, height=600)
fig.write_image(uniaxial_jpg_path, width=1200, height=600)

## Task 3

Compute the <b>quality functions</b> (numerical values) of the fitted model (in Task 2) for the equibiaxial and the 
planar extensions. Report the values!

In [113]:
error_value_P_B = quality_function(best_params, lambda_exp_equibiaxial, P_exp_equibiaxial, P_B) * 100
print(f"\nQuality for P_B: {error_value_P_B:.4f} %")

# Calculate the quality for P_P
error_value_P_P = quality_function(best_params, lambda_exp_planar, P_exp_planar, P_P) * 100
print(f"Quality for P_P: {error_value_P_P:.4f} %")


Quality for P_B: 9.4568 %
Quality for P_P: 3.3923 %


## Task 4

Visualize the model accuracy by <b>plotting the model solutions </b> for the stresses in each test using the fitted 
material parameters. Use separate figures for the tests. Use the plot range defined in Task 1. Write a few sentences (your 
<b>personal conclusions and observation</b>) about the accuracy of the fitted model.

In [114]:
# Calculate the quality for P_B
error_value_P_B = quality_function(best_params, lambda_exp_equibiaxial, P_exp_equibiaxial, P_B) * 100
print(f"\nQuality for P_B: {error_value_P_B:.4f} %")

# Calculate the quality for P_P
error_value_P_P = quality_function(best_params, lambda_exp_planar, P_exp_planar, P_P) * 100
print(f"Quality for P_P: {error_value_P_P:.4f} %")

# Plot the equibiaxial stress
fig_equibiaxial = go.Figure()

# Experimental data
fig_equibiaxial.add_trace(go.Scatter(x=lambda_exp_equibiaxial, y=P_exp_equibiaxial, mode='lines+markers', name='Experimental (Equibiaxial)', line=dict(color='red', dash='dash')))

# Simulated data using the best method
simulated_equibiaxial_stress = calculate_stress_equibiaxial(lambda_exp_equibiaxial, *best_params)
fig_equibiaxial.add_trace(go.Scatter(x=lambda_exp_equibiaxial, y=simulated_equibiaxial_stress, mode='lines+markers', name=f'Simulated Equibiaxial', line=dict(color='blue')))

fig_equibiaxial.update_layout(
    showlegend=True,
    plot_bgcolor='white',
    xaxis=dict(title='Stretch [1]', gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='black'),
    yaxis=dict(title='Engineering stress [MPa]', gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='black'),
    annotations=[
        dict(text=latex_annotations_mu1, x=0.07, y=0.97, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
        dict(text=latex_annotations_mu2, x=0.20, y=0.97, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
        dict(text=latex_annotations_alpha1, x=0.07, y=0.89, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
        dict(text=latex_annotations_alpha2, x=0.20, y=0.89, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
    ],
    shapes=[
        dict(
            type='rect',
            xref='paper',
            yref='paper',
            x0=0.055,
            y0=0.82,
            x1=0.315,
            y1=1,
            fillcolor='white',
            opacity=1,
            layer='below',
            line=dict(color='black', width=2)
        ),
        dict(
            type='rect',
            xref='paper',
            yref='paper',
            x0=0.055,
            y0=0.75,
            x1=0.315,
            y1=0.82,
            fillcolor='white',
            opacity=1,
            layer='below',
            line=dict(color='black', width=2)
        ),
    ]
)

fig_equibiaxial.add_annotation(
    text=f"Quality: {error_value_P_B:.4f} %",
    x=0.11, y=0.81,
    xref='paper', yref='paper',
    showarrow=False,
    align='left',
    font=dict(family='Arial, sans-serif')
)

fig_equibiaxial.show()

fig_planar = go.Figure()

fig_planar.add_trace(go.Scatter(x=lambda_exp_planar, y=P_exp_planar, mode='lines+markers', name='Experimental (Planar)', line=dict(color='red', dash='dash')))

# Simulated data using the best method
simulated_planar_stress = calculate_stress_planar(lambda_exp_planar, *best_params)
fig_planar.add_trace(go.Scatter(x=lambda_exp_planar, y=simulated_planar_stress, mode='lines+markers', name=f'Simulated Planar', line=dict(color='blue')))

# Layout and annotations
fig_planar.update_layout(
    #title='Experimental vs. Simulated Planar Stress',
    showlegend=True,
    plot_bgcolor='white',
    xaxis=dict(title='Stretch [1]', gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='black'),
    yaxis=dict(title='Engineering stress [MPa]', gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='black'),
    annotations=[
        dict(text=latex_annotations_mu1, x=0.07, y=0.97, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
        dict(text=latex_annotations_mu2, x=0.20, y=0.97, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
        dict(text=latex_annotations_alpha1, x=0.07, y=0.89, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
        dict(text=latex_annotations_alpha2, x=0.20, y=0.89, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
    ],
    shapes=[
        dict(
            type='rect',
            xref='paper',
            yref='paper',
            x0=0.055,
            y0=0.82,
            x1=0.315,
            y1=1,
            fillcolor='white',
            opacity=1,
            layer='below',
            line=dict(color='black', width=2)
        ),
        dict(
            type='rect',
            xref='paper',
            yref='paper',
            x0=0.055,
            y0=0.75,
            x1=0.315,
            y1=0.82,
            fillcolor='white',
            opacity=1,
            layer='below',
            line=dict(color='black', width=2)
        ),
    ]
)

fig_planar.add_annotation(
    text=f"Quality: {error_value_P_P:.4f} %",
    x=0.11, y=0.81,
    xref='paper', yref='paper',
    showarrow=False,
    align='left',
    font=dict(family='Arial, sans-serif')
)

fig_planar.show()


Quality for P_B: 9.4568 %
Quality for P_P: 3.3923 %


In [115]:
# Save the equibiaxial plot as HTML, PDF, and JPG
equibiaxial_html_path = os.path.join(output_folder, "equibiaxial_plot.html")
equibiaxial_pdf_path = os.path.join(output_folder, "equibiaxial_plot.pdf")
equibiaxial_jpg_path = os.path.join(output_folder, "equibiaxial_plot.jpg")

fig_equibiaxial.write_html(equibiaxial_html_path)
pio.write_image(fig_equibiaxial, equibiaxial_pdf_path, width=1000, height=600)
fig_equibiaxial.write_image(equibiaxial_jpg_path, width=900, height=600)

# Save the planar plot as HTML, PDF, and JPG
planar_html_path = os.path.join(output_folder, "planar_plot.html")
planar_pdf_path = os.path.join(output_folder, "planar_plot.pdf")
planar_jpg_path = os.path.join(output_folder, "planar_plot.jpg")

fig_planar.write_html(planar_html_path)
pio.write_image(fig_planar, planar_pdf_path, width=1000, height=600)
fig_planar.write_image(planar_jpg_path, width=900, height=600)

In [116]:
# Print the values in LaTeX format using IPython.display
display(Markdown(f"Uniaxial fit parameters"))
display(Markdown(f"\n$\\mu_1: {best_params[0]:.4f}$"))
display(Markdown(f"$\\mu_2: {best_params[1]:.4f}$"))
display(Markdown(f"$\\alpha_1: {best_params[2]:.4f}$"))
display(Markdown(f"$\\alpha_2: {best_params[3]:.4f}$"))

Uniaxial fit parameters


$\mu_1: 10.5544$

$\mu_2: 2.1012$

$\alpha_1: -0.7732$

$\alpha_2: 4.9987$

## Task 5

Obtain the material parameters by <b>fitting the model simultaneously to all measurements </b>. Consequently, the 
quality function is the mean of the individual quality values: $𝑄 = \frac{𝑄_𝑈 +𝑄_𝐵 + 𝑄_𝑃}{3}$. The parameter-fitting is acceptable if $𝑄 < 5$ %. Report the values of the fitted parameters and $𝑄,~𝑄_𝑈,~𝑄_𝐵,~𝑄_𝑃$.

In [117]:
# Define the quality function for all measurements
def total_quality_function(params, lambdas, P_exp_uniaxial, P_exp_equibiaxial, P_exp_planar):
    # Extract parameters
    mu1, mu2, alpha1, alpha2 = params
    
    # Calculate individual quality values
    Q_U = quality_function([mu1, mu2, alpha1, alpha2], lambda_exp_uniaxial, P_exp_uniaxial, P_U)
    Q_B = quality_function([mu1, mu2, alpha1, alpha2], lambda_exp_equibiaxial, P_exp_equibiaxial, P_B)
    Q_P = quality_function([mu1, mu2, alpha1, alpha2], lambda_exp_planar, P_exp_planar, P_P)
    
    # Calculate total quality as the mean of individual qualities
    Q_total = (Q_U + Q_B + Q_P) / 3.0
    
    return Q_total

# Initial guess
mu1 = 0.1
mu2 = 0.1
alpha1 = -2
alpha2 = 1

# Initial parameters for optimization
initial_params = [mu1, mu2, alpha1, alpha2]

# Set the optimization methods
methods = ['Nelder-Mead', 'BFGS', 'L-BFGS-B', 'Powell', 'COBYLA']

# Initialize variables to store the minimum error value and the corresponding method
min_error_value_total = float('inf')
best_method_total = None
best_params_total = None

# Loop through each optimization method
for method in methods:
    # Set the maximum number of iterations
    max_iter = 15000
    
    # Perform optimization
    result_optimize = minimize(total_quality_function, initial_params, args=(lambda_exp_uniaxial, P_exp_uniaxial, P_exp_equibiaxial, P_exp_planar), method=method, options={'maxiter': max_iter})
    optimal_params = result_optimize.x
    
    # Calculate total error value
    error_value_total = total_quality_function(optimal_params, lambda_exp_uniaxial, P_exp_uniaxial, P_exp_equibiaxial, P_exp_planar) * 100
    
    # Update minimum error value and best method
    if error_value_total < min_error_value_total:
        min_error_value_total = error_value_total
        best_method_total = method
        best_params_total = optimal_params

    # Print error value
    print(f"\nOptimal Parameters ({method}, Total): {optimal_params}")
    print(f"Total Error Value ({method}, Total): {error_value_total:.4f} % with {max_iter} iterations")

# Print the best method
print(f"\nBest Method: {best_method_total} (Error Value: {min_error_value_total:.4f} %)")
print(f"Optimal Parameters for the Best Method (total): {best_params_total}")


Optimal Parameters (Nelder-Mead, Total): [ 1.97175181 11.01756888  5.045279   -0.96092251]
Total Error Value (Nelder-Mead, Total): 3.4626 % with 15000 iterations

Optimal Parameters (BFGS, Total): [11.01752602  1.9717665  -0.9609219   5.04527311]
Total Error Value (BFGS, Total): 3.4626 % with 15000 iterations

Optimal Parameters (L-BFGS-B, Total): [11.01743523  1.97180688 -0.96093718  5.04525032]
Total Error Value (L-BFGS-B, Total): 3.4626 % with 15000 iterations

Optimal Parameters (Powell, Total): [11.00988509  1.96443403 -0.96619812  5.04837469]
Total Error Value (Powell, Total): 3.4653 % with 15000 iterations

Optimal Parameters (COBYLA, Total): [10.86969811  2.03146934 -0.97021645  5.01789347]
Total Error Value (COBYLA, Total): 3.4724 % with 15000 iterations

Best Method: BFGS (Error Value: 3.4626 %)
Optimal Parameters for the Best Method (total): [11.01752602  1.9717665  -0.9609219   5.04527311]


In [118]:
# Calculate quality values for the best method
Q_U_best = quality_function(best_params_total, lambda_exp_uniaxial, P_exp_uniaxial, P_U) * 100
Q_B_best = quality_function(best_params_total, lambda_exp_equibiaxial, P_exp_equibiaxial, P_B) * 100
Q_P_best = quality_function(best_params_total, lambda_exp_planar, P_exp_planar, P_P) * 100

# Report quality values for each measurement
print(f"\nQuality for P_U (Best Method - Total): {Q_U_best:.4f} %")
print(f"Quality for P_B (Best Method - Total): {Q_B_best:.4f} %")
print(f"Quality for P_P (Best Method - Total): {Q_P_best:.4f} %")


Quality for P_U (Best Method - Total): 4.2447 %
Quality for P_B (Best Method - Total): 3.0041 %
Quality for P_P (Best Method - Total): 3.1389 %


In [119]:
# Print the values in LaTeX format using IPython.display
display(Markdown("Total fit parameters"))
display(Markdown(f"\n$\\mu_1: {best_params_total[0]:.4f}$"))
display(Markdown(f"$\\mu_2: {best_params_total[1]:.4f}$"))
display(Markdown(f"$\\alpha_1: {best_params_total[2]:.4f}$"))
display(Markdown(f"$\\alpha_2: {best_params_total[3]:.4f}$"))

Total fit parameters


$\mu_1: 11.0175$

$\mu_2: 1.9718$

$\alpha_1: -0.9609$

$\alpha_2: 5.0453$

In [120]:
np.mean((((best_params - best_params_total) / best_params * 100)**2)**(1/2))

8.941383426874651

## Task 6

Visualize the model accuracy by <b>plotting the model solutions</b> for the stresses in each test using the fitted 
material parameters in Task 5. Use separate figures for the tests. Use the plot range defined in Task 1. Write a few 
sentences (your personal conclusions and observation) about the accuracy of the fitted model.

In [121]:
latex_annotations_mu1_total = f"$\\mu_1: {best_params_total[0]:.4f}$"
latex_annotations_mu2_total  = f"$\\mu_2: {best_params_total[1]:.4f}$"
latex_annotations_alpha1_total  = f"$\\alpha_1: {best_params_total[2]:.4f}$"
latex_annotations_alpha2_total  = f"$\\alpha_2: {best_params_total[3]:.4f}$"

# Function to generate plots for each test
def generate_and_save_plot(lambda_exp, P_exp, P_function, stress_type):
    # Calculate the quality for the test
    error_value = quality_function(best_params_total, lambda_exp, P_exp, P_function) * 100
    print(f"\nQuality for {stress_type}: {error_value:.4f} %")

    # Plot the stress
    fig = go.Figure()

    # Experimental data
    fig.add_trace(go.Scatter(x=lambda_exp, y=P_exp, mode='lines+markers', name=f'Experimental ({stress_type})', line=dict(color='red', dash='dash')))

    # Simulated data using the total fitted parameters
    simulated_stress = P_function(lambda_exp, *best_params_total)
    fig.add_trace(go.Scatter(x=lambda_exp, y=simulated_stress, mode='lines+markers', name=f'Simulated {stress_type}', line=dict(color='blue')))

    # Layout and annotations
    fig.update_layout(
        #title=f'Experimental vs. Simulated {stress_type} Stress',
        showlegend=True,
        plot_bgcolor='white',
        xaxis=dict(title='Stretch [1]', gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='black'),
        yaxis=dict(title='Engineering stress [MPa]', gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='black'),
        annotations=[
            dict(text=latex_annotations_mu1_total, x=0.07, y=0.97, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
            dict(text=latex_annotations_mu2_total, x=0.20, y=0.97, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
            dict(text=latex_annotations_alpha1_total, x=0.07, y=0.89, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
            dict(text=latex_annotations_alpha2_total, x=0.20, y=0.89, xref='paper', yref='paper', showarrow=False, align='left', font=dict(family='Arial, sans-serif')),
        ],
        shapes=[
            dict(
                type='rect',
                xref='paper',
                yref='paper',
                x0=0.055,
                y0=0.82,
                x1=0.315,
                y1=1,
                fillcolor='white',
                opacity=1,
                layer='below',
                line=dict(color='black', width=2)
            ),
            dict(
                type='rect',
                xref='paper',
                yref='paper',
                x0=0.055,
                y0=0.75,
                x1=0.315,
                y1=0.82,
                fillcolor='white',
                opacity=1,
                layer='below',
                line=dict(color='black', width=2)
            ),
        ]
    )

    fig.add_annotation(
        text=f"Quality: {error_value:.4f} %",
        x=0.11, y=0.81,
        xref='paper', yref='paper',
        showarrow=False,
        align='left',
        font=dict(family='Arial, sans-serif')
    )
    # Save the plot
    save_folder = "Cont_mecha_2HW"
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    html_path = os.path.join(save_folder, f"{stress_type.lower()}_plot_total.html")
    pdf_path = os.path.join(save_folder, f"{stress_type.lower()}_plot_total.pdf")
    jpg_path = os.path.join(save_folder, f"{stress_type.lower()}_plot_total.jpg")

    fig.write_html(html_path)
    pio.write_image(fig, pdf_path, width=1000, height=600)
    fig.write_image(jpg_path, width=900, height=600)

    # Display the plot
    fig.show()

# Generate and save plots for each test
generate_and_save_plot(lambda_exp_equibiaxial, P_exp_equibiaxial, calculate_stress_equibiaxial, 'Equibiaxial')
generate_and_save_plot(lambda_exp_planar, P_exp_planar, calculate_stress_planar, 'Planar')
generate_and_save_plot(lambda_exp_uniaxial, P_exp_uniaxial, calculate_stress_uniaxial, 'Uniaxial')


Quality for Equibiaxial: 3.0041 %



Quality for Planar: 3.1389 %



Quality for Uniaxial: 4.2447 %


## Task 7

The homogeneous displacement field in the material is given with the components of the material displacement 
field. The displacement components are given in [mm] if the coordinates are substituted in [mm]. The material is in a 
plane stress state, i.e. $\sigma_𝑧 = \tau_{𝑥𝑧} = \tau_{𝑦𝑧} = 0.$ <b>Determine the unknown scalar 𝑘</b> included in the displacement field using the 
incompressibility constraint!

In [152]:
# Define symbolic variables
X, Y, Z, k = symbols('X Y Z k')

# Displacement equations
ux_equation = 2 - k * Y
uy_equation = -0.6 * X + 0.1 * Y
uz_equation = -2

# Create the displacement vector U as a column vector
U = Matrix([ux_equation, uy_equation, uz_equation])

# Calculate the gradient of U (F = Grad U)
F = U.jacobian([X, Y, Z]) + eye(3)
F

Matrix([
[   1,  -k, 0],
[-0.6, 1.1, 0],
[   0,   0, 1]])

In [153]:
# Calculate the determinant of F (J = det(F))
J = det(F)

# Solve for the parameter k such that J = 1
equation = Eq(J, 1)
solutions = solve(equation, k)
sol = solutions[0]

In [154]:
# Print the values in LaTeX format using IPython.display
display(Markdown(f"Displacement vector :\n${latex(U)}$"))
display(Markdown(f"\nF matrix: ${latex(F)}$"))
display(Markdown(f"\nDeterminant $J$: \n${latex(J)}$"))
display(Markdown(f"\nParameter $k$ (solution for $J = 1$): \n$k = {latex(sol)}$"))

Displacement vector :
$\left[\begin{matrix}- Y k + 2\\- 0.6 X + 0.1 Y\\-2\end{matrix}\right]$


F matrix: $\left[\begin{matrix}1 & - k & 0\\-0.6 & 1.1 & 0\\0 & 0 & 1\end{matrix}\right]$


Determinant $J$: 
$1.1 - 0.6 k$


Parameter $k$ (solution for $J = 1$): 
$k = 0.166666666666667$

## Task 8

Determine the <b>principal stretches</b> and the normalized principal Eulerian directions.

In [155]:
F.subs(k,sol)

Matrix([
[   1, -0.166666666666667, 0],
[-0.6,                1.1, 0],
[   0,                  0, 1]])

In [156]:
C = F.transpose() * F
C_val = C.subs(k, sol)

b = F*F.transpose()
b_val = b.subs(k, sol)
b_val.applyfunc(lambda x: sp.N(x, 4))

Matrix([
[  1.028, -0.7833,   0],
[-0.7833,    1.57,   0],
[      0,       0, 1.0]])

In [158]:
(eigenVALS, eigenVECS) = np.linalg.eig(np.array(b_val, dtype=float))
eigenVALS_root = eigenVALS**(1/2)

## Task 9

Compute the components of the matrix of the <b>Cauchy stress</b> tensor using the fitted material parameters in Task 
5. Determine the <b>Mises equivalent stress</b>.

In [160]:
# Define symbols
mu1, mu2, alpha1, alpha2, lamb = sp.symbols('mu1 mu2 alpha1 alpha2 lamb')

# Compute the partial derivatives
partial_W_partial_lambda1 = (2*(mu1/alpha1 * lamb**(alpha1-1) + mu2/alpha2 * lamb**(alpha2-1))).subs({mu1: best_params_total[0], mu2: best_params_total[1], alpha1: best_params_total[2], alpha2: best_params_total[3], lamb: eigenVALS_root[0]})
partial_W_partial_lambda2 = (2*(mu1/alpha1 * lamb**(alpha1-1) + mu2/alpha2 * lamb**(alpha2-1))).subs({mu1: best_params_total[0], mu2: best_params_total[1], alpha1: best_params_total[2], alpha2: best_params_total[3], lamb: eigenVALS_root[1]})
partial_W_partial_lambda3 = (2*(mu1/alpha1 * lamb**(alpha1-1) + mu2/alpha2 * lamb**(alpha2-1))).subs({mu1: best_params_total[0], mu2: best_params_total[1], alpha1: best_params_total[2], alpha2: best_params_total[3], lamb: eigenVALS_root[2]})

In [162]:
sigma_new = eigenVALS_root[0] * partial_W_partial_lambda1 * sp.Matrix(eigenVECS[:,0]) * sp.Matrix(eigenVECS[:,0]).T + eigenVALS_root[1] * partial_W_partial_lambda2 * sp.Matrix(eigenVECS[:,1]) * sp.Matrix(eigenVECS[:,1]).T + eigenVALS_root[2] * partial_W_partial_lambda3 * sp.Matrix(eigenVECS[:,2]) * sp.Matrix(eigenVECS[:,2]).T
sigma_new

Matrix([
[-25.3938375804359, -10.4615353513333,                0],
[-10.4615353513333, -18.1523776493002,                0],
[                0,                 0, -22.149528954286]])

In [172]:
sp.Matrix(eigenVECS[:,2])

Matrix([
[  0],
[  0],
[1.0]])

In [164]:
s_new = sigma_new - 1/3*(sigma_new[0,0] + sigma_new[1,1] + sigma_new[2,2]) * eye(3)
p = -s_new[2,2]
s_new

Matrix([
[-3.49525618576188, -10.4615353513333,                  0],
[-10.4615353513333,  3.74620374537383,                  0],
[                0,                 0, -0.250947559611951]])

In [165]:
s_new_full = s_new + eye(3) *p
s_new_full

Matrix([
[-3.24430862614993, -10.4615353513333, 0],
[-10.4615353513333,  3.99715130498578, 0],
[                0,                 0, 0]])

In [166]:
ss = s_new*s_new.transpose()
mises_eq = sp.sqrt(3/2 * (ss[0,0] + ss[1,1] + ss[2,2]))

In [167]:
ss

Matrix([
[ 121.660537711304, -2.62529676621123,                  0],
[-2.62529676621123,   123.47776440905,                  0],
[                0,                 0, 0.0629746776751939]])

In [168]:
print(f"Mises equivalent stress: {mises_eq:.4f} [MPa]")

Mises equivalent stress: 19.1782 [MPa]
